<a href="https://colab.research.google.com/github/VOTUANANH01/DS104_Distributed_and_parallel_computing/blob/main/ass1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/VOTUANANH01/DS104_Distributed_and_parallel_computing

Cloning into 'DS104_Distributed_and_parallel_computing'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 37 (delta 15), reused 12 (delta 1), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [ ]:
!pip install pyspark==3.0.1
%cd /usr/local/lib/python3.7/dist-packages/pyspark/jars
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar
%cd '/content/'
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar
!jar xf /content/graphframes-0.8.1-spark3.0-s_2.12.jar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 204.2 MB 34 kB/s 
     |████████████████████████████████| 198 kB 23.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=3fbfa63aaec5278c98e27a8c7f79928e3d0fa78f043b66269feb76c9cdcb6bb1
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark
/usr/local/lib/python3.7/dist-packages/pyspark/jars
--2022-09-27 09:14:10--  https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 18.154.110.54, 18.154.110.122, 18.154.110.33, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|18.154.110.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248133 (242K) [binary/octet-

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as f

spark=SparkSession.builder.getOrCreate()

#1. Load data from flightdata2018.json and airports.json


In [ ]:
flightDF=spark.read.json('/content/DS104_Distributed_and_parallel_computing/ass1/flightdata2018.json')


In [ ]:
flightEdges=flightDF.withColumnRenamed("depdelay","delay")\
  .select("id","fldate","month","dofW","carrier","src","dst","crsdephour","crsdeptime"\
  ,"delay","crsarrtime","arrdelay","crselapsedtime","dist")

In [ ]:
flightEdges.printSchema()

root
 |-- id: string (nullable = true)
 |-- fldate: string (nullable = true)
 |-- month: long (nullable = true)
 |-- dofW: long (nullable = true)
 |-- carrier: string (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- crsdephour: long (nullable = true)
 |-- crsdeptime: long (nullable = true)
 |-- delay: double (nullable = true)
 |-- crsarrtime: long (nullable = true)
 |-- arrdelay: double (nullable = true)
 |-- crselapsedtime: double (nullable = true)
 |-- dist: double (nullable = true)



In [ ]:
flightEdges.show(5)

+--------------------+----------+-----+----+-------+---+---+----------+----------+-----+----------+--------+--------------+-----+
|                  id|    fldate|month|dofW|carrier|src|dst|crsdephour|crsdeptime|delay|crsarrtime|arrdelay|crselapsedtime| dist|
+--------------------+----------+-----+----+-------+---+---+----------+----------+-----+----------+--------+--------------+-----+
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|         9|       850|  0.0|      1116|     0.0|         146.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        11|      1122|  8.0|      1349|     0.0|         147.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        14|      1356|  9.0|      1623|     0.0|         147.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        16|      1620|  0.0|      1851|     3.0|         151.0|946.0|
|ATL_BOS_2018-01-0...|2018-01-01|    1|   1|     DL|ATL|BOS|        19|      1940|  6.0|  

In [ ]:
airportVertices=spark.read.json("/content/DS104_Distributed_and_parallel_computing/Slide 5-6 data/airports.json")

In [ ]:
airportVertices.show()

+-------------+-------+-----+---+
|         City|Country|State| id|
+-------------+-------+-----+---+
|      Chicago|    USA|   IL|ORD|
|     New York|    USA|   NY|LGA|
|       Boston|    USA|   MA|BOS|
|      Houston|    USA|   TX|IAH|
|       Newark|    USA|   NJ|EWR|
|       Denver|    USA|   CO|DEN|
|        Miami|    USA|   FL|MIA|
|San Francisco|    USA|   CA|SFO|
|      Atlanta|    USA|   GA|ATL|
|       Dallas|    USA|   TX|DFW|
|    Charlotte|    USA|   NC|CLT|
|  Los Angeles|    USA|   CA|LAX|
|      Seattle|    USA|   WA|SEA|
+-------------+-------+-----+---+



In [ ]:
#create graphframe
from graphframes import *
flightGraph=GraphFrame(airportVertices,flightEdges)
print(flightGraph)

GraphFrame(v:[id: string, City: string ... 2 more fields], e:[src: string, dst: string ... 12 more fields])


##1.1. Show the most important airports according to pageRank (resetProbability=0.15, maxIter=10). The result should be displayed in descending order of pagerank.

In [ ]:
ranks=flightGraph.pageRank(resetProbability = 0.15,maxIter=5)

ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).show(20)

+-------------+-------+-----+---+------------------+
|         City|Country|State| id|          pagerank|
+-------------+-------+-----+---+------------------+
|      Chicago|    USA|   IL|ORD|1.4212550221589475|
|      Atlanta|    USA|   GA|ATL| 1.339101482426853|
|  Los Angeles|    USA|   CA|LAX| 1.200976131283591|
|       Dallas|    USA|   TX|DFW|1.1270531678567586|
|       Denver|    USA|   CO|DEN|1.0591136061016508|
|San Francisco|    USA|   CA|SFO|1.0246282632093013|
|     New York|    USA|   NY|LGA|0.9447562478927347|
|       Boston|    USA|   MA|BOS|0.8775066074875163|
|       Newark|    USA|   NJ|EWR|0.8731036205114817|
|        Miami|    USA|   FL|MIA|0.8507878300790072|
|      Houston|    USA|   TX|IAH|0.8350718288420121|
|    Charlotte|    USA|   NC|CLT|0.8049000935422239|
|      Seattle|    USA|   WA|SEA|0.6417460986079204|
+-------------+-------+-----+---+------------------+



##1.2 Show the average delay of each Scheduled Depart Hour (crsdephour) from LGA airport. The result should be displayed in descending order of average delay

In [ ]:
flightGraph.edges.filter("src='LGA' and delay > 1" ) \
.groupby("crsdephour").avg("delay").sort(desc("avg(delay)")).show()

+----------+------------------+
|crsdephour|        avg(delay)|
+----------+------------------+
|        21| 58.51034482758621|
|        19| 58.48443579766537|
|        18| 58.30467289719626|
|        15| 56.19718309859155|
|        22|52.986666666666665|
|        20| 52.81941309255079|
|        17| 49.28086419753087|
|        16| 43.59234234234234|
|        13| 42.80222222222222|
|        12| 42.41463414634146|
|        14|40.632603406326034|
|        11| 38.05428571428571|
|         9| 37.07849829351536|
|         7| 36.63453815261044|
|        10|  34.9458762886598|
|         8| 34.69613259668508|
|         6| 30.15813953488372|
|         5| 27.73170731707317|
+----------+------------------+



##1.3. Find all destinations that does not have direct flight from LGA but can be reached with one transit. The result should be display in the ascending order of the id of destination and transit airport.

In [ ]:
flight_route_count=flightGraph.edges.groupBy("src","dst").count().orderBy(col("count").desc())

In [ ]:
sub_graph=GraphFrame(flightGraph.vertices, flight_route_count)

In [ ]:
motif= sub_graph.find("(a)-[]->(b);(b)-[]->(c);!(a)-[]->(c)")\
  .filter("c.id !=a.id")
  # .orderBy(f.col("c.id").asc(),f.col("b.id").asc())\
  # .selectExpr("a.id as origin","c.id as destination","b.id as transit")\

In [ ]:
motif= flightGraph.find("(a)-[]->(b);(b)-[]->(c);!(a)-[]->(c)")\
  .filter("c.id !=a.id")
  # .orderBy(f.col("c.id").asc(),f.col("b.id").asc())\
  # .selectExpr("a.id as origin","c.id as destination","b.id as transit")\

In [ ]:
motif.show(1)

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
|[New York, USA, N...|[Denver, USA, CO,...|[San Francisco, U...|
+--------------------+--------------------+--------------------+
only showing top 1 row



In [ ]:
motif.filter("a.id == 'LGA'")\
.orderBy(f.col("c.id").asc(),f.col("b.id").asc())\
.selectExpr("a.id as origin","c.id as destination","b.id as transit")\
.show(32)

+------+-----------+-------+
|origin|destination|transit|
+------+-----------+-------+
|   LGA|        EWR|    ATL|
|   LGA|        EWR|    BOS|
|   LGA|        EWR|    CLT|
|   LGA|        EWR|    DEN|
|   LGA|        EWR|    DFW|
|   LGA|        EWR|    IAH|
|   LGA|        EWR|    MIA|
|   LGA|        EWR|    ORD|
|   LGA|        LAX|    ATL|
|   LGA|        LAX|    BOS|
|   LGA|        LAX|    CLT|
|   LGA|        LAX|    DEN|
|   LGA|        LAX|    DFW|
|   LGA|        LAX|    IAH|
|   LGA|        LAX|    MIA|
|   LGA|        LAX|    ORD|
|   LGA|        SEA|    ATL|
|   LGA|        SEA|    BOS|
|   LGA|        SEA|    CLT|
|   LGA|        SEA|    DEN|
|   LGA|        SEA|    DFW|
|   LGA|        SEA|    IAH|
|   LGA|        SEA|    MIA|
|   LGA|        SEA|    ORD|
|   LGA|        SFO|    ATL|
|   LGA|        SFO|    BOS|
|   LGA|        SFO|    CLT|
|   LGA|        SFO|    DEN|
|   LGA|        SFO|    DFW|
|   LGA|        SFO|    IAH|
|   LGA|        SFO|    MIA|
|   LGA|      

##1.4. Computes shortest paths from each airport to LGA.

In [ ]:
shortestpath=flightGraph.shortestPaths(landmarks=['LGA'])
shortestpath.select("id","distances").show()

+---+----------+
| id| distances|
+---+----------+
|IAH|[LGA -> 1]|
|CLT|[LGA -> 1]|
|LAX|[LGA -> 2]|
|DEN|[LGA -> 1]|
|DFW|[LGA -> 1]|
|SFO|[LGA -> 2]|
|LGA|[LGA -> 0]|
|ORD|[LGA -> 1]|
|MIA|[LGA -> 1]|
|SEA|[LGA -> 2]|
|ATL|[LGA -> 1]|
|BOS|[LGA -> 1]|
|EWR|[LGA -> 2]|
+---+----------+

